In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy.spatial.distance import cosine
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())
import operator

0it [00:00, ?it/s]


In [2]:
with open("train.csv", "r") as f:
    train_text = f.read()
    
rows = train_text.split("\n")
rows_list = []

for i in rows:
    
    data = i.split("|")
    
    if len(data) == 3:
        user_id = data[0]
        movie_name = data[1]
        watch_time = float(data[2])
    
        rows_list.append([user_id, movie_name, watch_time])
        
train = pd.DataFrame(rows_list)
train.columns = ["user_id", "title", "watch_time"]

test = pd.read_csv("test.csv")

with open("movies_meta.csv", "r") as f:
    movies_meta = f.read()

movies = movies_meta.split("\n")[1:]

movies_df = pd.DataFrame()

movies_list = []

for movie in movies:
    
    data = movie.split("|")

    if len(data) == 4:

        movie_name = data[0]
        genre = data[1]

        langs = data[2].split(",")

        movie_length = float(data[3])

        for lang in langs:
            movies_list.append([movie_name, genre, lang, movie_length])

movies_df = pd.DataFrame(movies_list)
movies_df.columns = ["title", "movie_genre", "movie_language", "movie_length"]    

In [6]:
title_count = {}

def get_movie_counts(row):
        
    if row.title not in title_count:
        title_count[row.title] = 1
    else:
        title_count[row.title] += 1
        
train.progress_apply(get_movie_counts, axis = 1)

2000000it [04:12, 7930.77it/s] 


0          None
1          None
2          None
3          None
4          None
5          None
6          None
7          None
8          None
9          None
10         None
11         None
12         None
13         None
14         None
15         None
16         None
17         None
18         None
19         None
20         None
21         None
22         None
23         None
24         None
25         None
26         None
27         None
28         None
29         None
           ... 
1999970    None
1999971    None
1999972    None
1999973    None
1999974    None
1999975    None
1999976    None
1999977    None
1999978    None
1999979    None
1999980    None
1999981    None
1999982    None
1999983    None
1999984    None
1999985    None
1999986    None
1999987    None
1999988    None
1999989    None
1999990    None
1999991    None
1999992    None
1999993    None
1999994    None
1999995    None
1999996    None
1999997    None
1999998    None
1999999    None
Length: 2000000, dtype: 

In [9]:
popular_movies = [i[0] for i in sorted(title_count.items(), key=operator.itemgetter(1), reverse=True)[:20]]

In [11]:
res = {}

def get_similar_items(row):
    
    res[str(row.user_id)] = popular_movies

test.progress_apply(get_similar_items, axis = 1)

100%|██████████| 6285/6285 [00:00<00:00, 35196.84it/s]


0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
6255    None
6256    None
6257    None
6258    None
6259    None
6260    None
6261    None
6262    None
6263    None
6264    None
6265    None
6266    None
6267    None
6268    None
6269    None
6270    None
6271    None
6272    None
6273    None
6274    None
6275    None
6276    None
6277    None
6278    None
6279    None
6280    None
6281    None
6282    None
6283    None
6284    None
Length: 6285, dtype: object

In [13]:
import json
with open('res_popular.json', 'w') as fp:
    json.dump(res, fp)